# Problem

In our company we use SCRUM as a project management framework. With each new sprint we go through the process of prioritizing demands and allocating these demands to employees. We have teams with 3 or more data scientists, with different levels of seniority and tasks with varying complexities. 

>This scenario sometimes makes it difficult for us to allocate the right tasks to the right employees in a way that helps us complete all the sprint tasks.

# Solution

We can build an **optimization model to minimize the time used to complete demands**. This problem falls into the class of assignment problems, a special case of transportation problems. 

The optimization model will receive as input the estimated time that each employee would take to complete each of the tasks. The model will then find which task each employee should do in a way that minimizes task completion time. 


This problem falls into the class of assignment problems, a special case of transportation problems. The optimization model will receive as input the estimated time that each employee would take to complete each of the tasks. The model will then find which task each employee should do in a way that minimizes task completion time.

Each employee-task combination will be a decision variable whose coefficient will be the estimated time to complete the task (cost). As a constraint we need to ensure that each employee is allocated to only one task and each task is allocated to only one employee. As output we will have variables with a value of 0 or 1, and the variables with a value of 1 will be the combinations that minimize time. 

In this notebook we cover two different ways of doing this optimization:
- using scipy
- using pyomo

### Example

In [1]:
import pandas as pd

In [2]:
df = (
    pd.DataFrame([('mateus', 15, 10, 9), 
                  ('maria', 9, 15, 10), 
                  ('joao', 10, 12, 8)], columns=['employee', 'task_1', 'task_2', 'task_3'])
      .set_index('employee')
)

df

,task_1,task_2,task_3
employee,,,
mateus,15,10,9
maria,9,15,10
joao,10,12,8


In [3]:
cost_matrix = df.values

cost_matrix

array([[15, 10,  9],
       [ 9, 15, 10],
       [10, 12,  8]])

In this example the objective function would be:

$$\min Z = 15x_{11} + 10x_{12} + 9x_{13} + 9x_{21} + 15x_{22} + 10x_{23} + 10x_{31} + 12x_{32} + 8x_{33}$$

$$s.t.$$
$$x_{11} + x_{12} + x_{13} = 1$$
$$x_{21} + x_{22} + x_{23} = 1$$
$$x_{31} + x_{32} + x_{33} = 1$$

$$x_{11} + x_{21} + x_{31} = 1$$
$$x_{12} + x_{22} + x_{32} = 1$$
$$x_{13} + x_{23} + x_{33} = 1$$

where,
- $x_{11}$ = mateus-t1
- $x_{12}$ = mateus-t2
- $x_{13}$ = mateus-t3

- $x_{21}$ = maria-t1
- $x_{22}$ = maria-t2
- $x_{23}$ = maria-t3

- $x_{31}$ = joao-t1
- $x_{32}$ = joao-t2
- $x_{33}$ = joao-t3


In [4]:
def print_assignment(list_variables, n):

    count = 0
    for destination in range(n):
        sources = list(list_variables[count:count + n])
        count += n
        print(f'Task {destination + 1} assigned to Worker {sources.index(1.) + 1}')

### Scipy

In [5]:
from scipy.optimize import linprog
import numpy as np

def min_assignment_problem(cost_matrix: np.ndarray, method : str = 'highs'):

    '''
    This function is a generalization of the assignment problem.
    Given a cost matrix, and respecting the principle of balancing (quadratic cost matrix), this function builds a minimizer that 
    determines which source-output should be used to minimize the cost. 

    The return is the minimization cost of the function and a vector with binary values, where 1 = variable (origin-destination) of 
    that position is the designation that minimizes the cost.
    
    :param cost_matrix: Cost matrix (origin x destination)
    :type cost_matrix: numpy.ndarray

    :param method: Same scipy, more information on possible values -> 
                   https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html
    :type method: str
    '''

    n = cost_matrix.shape[0]

    if cost_matrix.shape[0] != cost_matrix.shape[1]:
        raise ValueError('The matrix must have the same number of rows and columns')
    
    list_inequalities = np.zeros(shape=((n + n), (n * n)))
    list_upper_bound = np.full((n + n), -1)
    
    # Constraints that ensure that a source will only be associated with one destination (1 worker -> 1 task)
    step = 0
    for source in range(n):    
        list_inequalities[source, step:step + n] = -1
        step += n
    
    # Constraints that ensure that the destination will only be associated with one source (1 task -> 1 worker)
    count = 0
    for destination in range(n, n + n):
        for source in range(count, (n * n), n):
            list_inequalities[destination, source] = -1
        count+=1
    
    res = linprog(cost_matrix.reshape(-1), A_ub=list_inequalities, b_ub=list_upper_bound, bounds=(0, None), method=method)

    return res.fun, res.x

In [6]:
fun, x = min_assignment_problem(cost_matrix)

print(fun, x)

27.0 [ 0.  1.  0.  1.  0. -0.  0. -0.  1.]


In [7]:
print_assignment(x, cost_matrix.shape[0])

Task 1 assigned to Worker 2
Task 2 assigned to Worker 1
Task 3 assigned to Worker 3


### Pyomo

In [8]:
from pyomo.environ import *

def min_assignment_problem(cost_matrix: np.ndarray, solver_name : str = 'glpk'):

    '''
    This function is a generalization of the assignment problem.
    Given a cost matrix, and respecting the principle of balancing (quadratic cost matrix), this function builds a minimizer that 
    determines which source-output should be used to minimize the cost. 

    The return is the minimization cost of the function and a vector with binary values, where 1 = variable (origin-destination) of 
    that position is the designation that minimizes the cost.
    
    :param cost_matrix: Cost matrix (origin x destination)
    :type cost_matrix: numpy.ndarray

    :param solver_name: Same pyomo SolverFactory, more information on possible values -> 
                   https://pyomo.readthedocs.io/en/stable/working_models.html
    :type solver_name: str
    '''

    n = cost_matrix.shape[0]

    if cost_matrix.shape[0] != cost_matrix.shape[1]:
        raise ValueError('The matrix must have the same number of rows and columns')

    model = ConcreteModel()

    model.x = VarList(within=NonNegativeReals)
    for i in range(n*n):
        model.x.add()

    model.obj = Objective(expr=sum([coef * model.x[idx] for coef, idx in zip(cost_matrix.reshape(-1), range(1, (n*n) + 1))]), 
                          sense=minimize)

    # Constraints that ensure that a source will only be associated with one destination (1 worker -> 1 task)
    model.constraint_one_source = ConstraintList()
    step = n + 1
    for source in range(1, n*n + 1, n):
        model.constraint_one_source.add(sum([model.x[idx] for idx in range(source, step)]) == 1)
        step += n

    # Constraints that ensure that the destination will only be associated with one source (1 task -> 1 worker)
    model.constraint_one_destination = ConstraintList()
    for destination in range(1, n + 1):
        model.constraint_one_destination.add(sum([model.x[idx] for idx in range(destination, n*n + 1, n)]) == 1)

    solver = SolverFactory(solver_name)
    solver.solve(model)
    
    return model.obj(), [model.x[idx]() for idx in range(1, n*n + 1)]

In [9]:
fun, x = min_assignment_problem(cost_matrix)

print(fun, x)

27.0 [0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]


In [10]:
print_assignment(x, cost_matrix.shape[0])

Task 1 assigned to Worker 2
Task 2 assigned to Worker 1
Task 3 assigned to Worker 3


## Some References:
- https://medium.com/aimonks/linear-programming-and-the-simplex-method-building-and-comparing-implementations-in-python-6e015c449409
- https://towardsdatascience.com/a-beginners-guide-to-linear-programming-and-the-simplex-algorithm-87db017e92b4
- https://towardsdatascience.com/some-thoughts-on-synergies-between-operations-research-and-machine-learning-921d78ed4bd5